## 기존의 xlsx 파일을 csv로 변환

In [22]:
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm

song_df = pd.read_excel('../song_data/song_df.xlsx', index_col=0)
artist_df = pd.read_excel('../song_data/artist_df.xlsx', index_col=0)
album_df = pd.read_excel('../song_data/album_df.xlsx', index_col=0)
song_artist_df = pd.read_excel('../song_data/song_artist_df.xlsx', index_col=0)
song_album_df = pd.read_excel('../song_data/song_album_df.xlsx', index_col=0)
song_genre_df = pd.read_excel('../song_data/song_genre_df.xlsx', index_col=0)
lyrics_df = pd.read_excel('../song_data/song_lyrics_df.xlsx', index_col=0)

### 장르 테이블 (gnere_id, genre_name) 생성

In [4]:
genre_df = pd.DataFrame(columns=['genre_id', 'genre_name'])

genre_list = [
    [101, '발라드'],
    [102, '포크/블루스'],
    [103, '성인가요/트로트'],
    [104, '록/메탈'],
    [105, '국내영화'],
    [106, '댄스'],
    [107, '국내드라마'],
    [108, '뉴에이지'],
    [109, '클래식'],
    [110, '크로스오버'],
    [111, '교향/관현악'],
    [112, '랩/힙합'],
    [113, '-'],
    [114, '재즈'],
    [115, '보컬재즈'],
    [116, '인디음악'],
    [117, 'R&B/Soul'],
    [118, '애시드/퓨전/팝'],
    [119, '애니메이션/웹툰'],
    [120, 'POP'],
    [121, '일렉트로니카'],
    [122, 'J-POP'],
    [123, 'CCM'],
    [124, '국내CCM'],
    [125, '워십'],
    [126, '찬송가'],
    [127, '게임'],
    [128, '키즈'],
    [129, '만화'],
    [130, '국외영화'],
    [131, '국내뮤지컬']
]

for x,y in genre_list:
    genre_df.loc[len(genre_df)] = [x, y]

genre_df.to_csv('../song_data/genre.csv')

### xlsx -> csv

In [23]:
song_df = song_df.drop_duplicates(['song_id']).reset_index(drop=True)
for i in tqdm(range(len(song_df))):
    title = song_df.loc[i, 'title']
    song_df.loc[i, 'title'] = title.replace(',', '&')
    # 19금 노래는 가사를 받아올 수 없으므로 nan
    if title[:3] == '19금':
        title = np.nan
        song_df.loc[i, 'title'] = title

# 19금 노래 제거 후 인덱스 리셋
song_df = song_df.dropna(axis=0).reset_index(drop=True)

# csv로 저장
song_df.to_csv('../song_data/song.csv')

  0%|          | 0/9278 [00:00<?, ?it/s]

In [20]:
song_artist_df = song_artist_df.drop_duplicates().reset_index(drop=True)
for i in range(song_artist_df):
    song_id = song_artist_df.loc[i, 'song_id']
    if not song_df[song_df.loc[:, 'song_id'] == song_id].any():
        song_artist_df.loc[i, 'song_id'] = np.nan
    

,song_id,title
0,52441,너에게로 또 다시
1,53060,솔아 솔아 푸르른 솔아
2,1017150,그 아픔까지 사랑한거야
3,53018,향기로운 추억 (응답하라 1988 삽입곡)
4,1859404,잊지 말아요
...,...,...
9273,30117119,홍연
9274,2795225,Dear (영화 '아저씨'삽입곡)
9275,469788,Don't Cry
9276,8115185,LIAR LIAR


In [28]:
song_id = 12312314

In [29]:
song_df[song_df.loc[:, 'song_id'] == song_id].any()

song_id    False
title      False
dtype: bool

In [26]:
song_artist_df.drop_duplicates()

,song_id,artist_id
0,52441,1128
1,53060,101085
2,1017150,6014
3,53018,1163
4,1859404,35
...,...,...
10080,30117119,947099
10081,2795225,404720
10082,469788,107224
10083,8115185,857994


In [16]:
title = song_df.loc[3353, 'title']
title[:3] == '19금'

True

In [53]:
song_lyrics_df = pd.DataFrame(columns=['song_id','title','lyrics'])

In [54]:
for i in range(len(song_df)):
    song_id, title = song_df.loc[i, ['song_id','title']]
    lyrics = lyrics_df[lyrics_df['song_id'] == song_id].lyrics.item()
    title = title.replace(',', '&')
    
    lyrics = lyrics.replace(',','_').replace('\n</div>', '').replace('</p>', '')
    if lyrics == 'None' or lyrics[:3] == '19금':
        lyrics = np.nan
    
    song_lyrics_df.loc[i, :] = [song_id, title, lyrics]

In [55]:
song_lyrics_df.to_excel('./song_lyrics_df.xlsx')
song_lyrics_df.to_csv('./song_lyrics_df.csv')

In [56]:
df = pd.read_csv('./song_lyrics_df.csv', index_col=0)
df

ParserError: Error tokenizing data. C error: Expected 4 fields in line 4378, saw 6


'미련없이 내뿜는 담배연기속에  \\n 아련히 떠오르는 그여인의 얼굴을 \\n 별마다 새겨보는 별마다 새겨보는  \\n 아~ 진고개 신사 \\n  \\n 헝클어진 머리를 쓸어 올리며 \\n 언젠가 불러주던 그여인의 노래를  \\n 소리없이 불러보던 소리없이 불러보던 \\n 아~ 진고개 신사 \n'